In [32]:
import os
import exifread
import pandas as pd

def convertir_valeur_exif(valeur):
    try:
        num, denom = str(valeur).split('/')
        return float(num) / float(denom)
    except ValueError:
        return None

In [28]:
def calculer_pdc(f, r, k, g):
    if f is None or k is None or g is None:
        return None
    f = f / 1000  
    g = g / 1000  
    r = r / 1000  
    try:
        pdc = (4 * f**2 * r * k * g * (g - f)) / (f**4 - 4 * r**2 * k**2 * (g - f)**2)
        return pdc
    except ZeroDivisionError as e:
        return None

In [31]:
chemin_dossier = "TP1_Donnee/TP1_Donnee/PdC"
donnees_photos = []

pdc_observé_liste = [0.0055, 0.007, 0.013, 0.0075, 0.009, 0.009]

for index, fichier in enumerate(os.listdir(chemin_dossier)):
    if fichier.lower().endswith((".jpg", ".jpeg")) and index < len(pdc_observé_liste):
        chemin_photo = os.path.join(chemin_dossier, fichier)
        with open(chemin_photo, 'rb') as f:
            tags = exifread.process_file(f)
            r = 0.030
            g = 1000
            ouverture = tags.get('EXIF FNumber')
            distance_focale = tags.get('EXIF FocalLength')
            if distance_focale and ouverture:
                distance_focale = convertir_valeur_exif(distance_focale)
                ouverture = convertir_valeur_exif(ouverture)
                pdc_calculé = calculer_pdc(distance_focale, r, ouverture, g)
            else:
                pdc_calculé = None

            donnees_photos.append({
                "Nom": fichier,
                "Ouverture": ouverture,
                "D_Focale": distance_focale,
                "PDC_calculé": pdc_calculé,
                "Pdc_observé": pdc_observé_liste[index],
            })

df = pd.DataFrame(donnees_photos)

df['Difference_PDC'] = df['PDC_calculé'] - df['Pdc_observé']

print(df)


            Nom  Ouverture  D_Focale  PDC_calculé  Pdc_observé  Difference_PDC
0  PICT3149.JPG        3.2  7.203125    -0.587952       0.0055       -0.593452
1  PICT3150.JPG        2.8  7.203125    -0.688818       0.0070       -0.695818
2  PICT3151.JPG        NaN  7.203125          NaN       0.0130             NaN
3  PICT3152.JPG        NaN  7.203125          NaN       0.0075             NaN
4  PICT3153.JPG        5.6  7.203125    -0.318793       0.0090       -0.327793
5  PICT3154.JPG        2.8  7.203125    -0.688818       0.0090       -0.697818
